In [1]:
from libs.graphs_and_points import create_graph, trading_points, setup_figure, graph_with_points
from libs.model_training import output_from_model
import pandas as pd
from ta.trend import SMAIndicator

In [3]:
types = ['apple', 'bitcoin', 'cez', 'zlato']
#models_paths = ['out_sorted/apple_9.3_slide_5.pt', 'out_sorted/bitcoin_1611.6_slide_7.pt', 'out_sorted/cez_140.2_slide_5.pt', 'out_sorted/zlato_29.3_slide_21.pt']
models_paths = ['data_konf/models/apple.pth', 'data_konf/models/bitcoin.pth', 'data_konf/models/cez.pth', 'data_konf/models/zlato.pth']
windows = [5, 7, 5, 21]
noise = False
temp = []
dates = []
targets = []

for i in range(4):
    data = pd.read_csv("data_konf/" + types[i] + "_spojeni_cele.csv")
    data.set_index("Date", inplace=True)
    dates.append(data.index.tolist())
    
    target = data[types[i] + "_Close"].values
    target = target[1:]
    target = pd.Series(target)
    targets.append(target)
    data = data[:-1]
    
    new_target = SMAIndicator(close=target, window=windows[i]).sma_indicator()
    new_target = new_target[windows[i]-1:]
    data = data[windows[i]-1:]
    
    results = output_from_model(models_paths[i], data, new_target, noise)

    y_true_train = results['train_true'].flatten()
    y_pred_train = results['train_pred'].flatten()
    y_true_test = results['test_true'].flatten()
    y_pred_test = results['test_pred'].flatten()
    temp.append([y_true_train, y_pred_train, y_true_test, y_pred_test])
table = pd.DataFrame(temp, columns=['train_true', 'train_pred', 'test_true', 'test_pred'], index=types)

dates_table = []
for i in range(4):
    train_size = len(table['train_true'][types[i]])
    test_size = len(table['test_true'][types[i]])
    
    dates_table.append([dates[i][:train_size], dates[i][-test_size:]])

In [20]:
tp = 'zlato'
temp = targets[3]

temp = temp[-len(table['test_true'][tp]):]

list_of_stocks = [temp, table['test_pred'][tp]]
names = ['True values', 'Predicted value']
axis = ['Date', 'Price']
colours = ['blue', 'red']
widths = [1, 1]
title = 'Prediction gold price'
font_size = 14
x_axes = dates_table[0][1]
plot_w = 1000
plot_h = 600
legend_x = "right"
legend_y = "bottom"
pos_x = 0.99
pos_y = 0.01

p_names = ["Sell point", "Buy point"]
p_colours = ["green", "red"]

In [21]:
down_points, up_points = trading_points(table['test_pred'][tp], 0.05)
fig1 = setup_figure(list_of_stocks, names, axis, colours, widths, title, font_size, x_axes, plot_w, plot_h, legend_x, legend_y, pos_x, pos_y)
graph_with_points(fig1, [up_points, down_points], temp.to_list(), x_axes, p_names, p_colours)

In [30]:
create_graph(list_of_stocks, names, axis, colours, widths, title, font_size, x_axes, plot_w, plot_h, legend_x, legend_y, pos_x, pos_y)

In [33]:
data = pd.read_csv("data_konf/" + "cez" + "_spojeni_cele.csv")
data.set_index("Date", inplace=True)
list_of_stocks = [data['cez_Close'].values]
names = ['Price of ČEZ stocks']
axis = ['Date', 'Price']
colours = ['blue']
widths = [2]
title = 'ČEZ stocks price'
font_size = 14
x_axes = data.index.tolist()
plot_w = 1000
plot_h = 600
legend_x = "right"
legend_y = "bottom"
pos_x = 0.99
pos_y = 0.02

In [ ]:
create_graph(list_of_stocks, names, axis, colours, widths, title, font_size, x_axes, plot_w, plot_h, legend_x, legend_y, pos_x, pos_y)